# There are 11 categories in CNA news

Crawl links on a category's first page.

中央社新聞爬蟲-11類別的新聞(只能爬第一頁的內容，沒有翻頁的功能)

In [1]:
# 11 news categories
news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
base_url = 'https://www.cna.com.tw/list/'

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta

In [7]:
# item id 抓時間 當中有夾帶廣告 沒有時間欄位，可以判斷之並將之跳離這一筆!

In [ ]:
%%time
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []

for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    full_url = base_url + url_short_name + '.aspx'
    print(full_url)
    req = requests.get(full_url)
    page = BeautifulSoup(req.text, 'lxml')
    #page.find('li',{'class':"noPadding myEpaperInList"}).decompose() #去除不要的一堆插入連結與文字

    # 抓新聞列表
    items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')

    serial_no = 1
    for j, item in enumerate(items):  #針對每一篇項目 抓其細節
        title = item.find('h2').text  #抓標題
        # print(title)
        link = item.find('a').get('href')  #抓連結
        #print(link)
        dt = item.find('div', {'class': "date"}).text  #抓日期時間
        #print(dt)
        titles.append(title)
        links.append(link)
        categories.append(news_categories[i])  #類別名稱紀錄起來

        #
        # 抓時間 當中有夾帶廣告 沒有時間欄位，可以判斷之並將之跳離這一筆!
        # 排除穿插廣告
        try:
            dt = item.find('div', {'class': "date"}).text  #抓日期時間
            # 排除穿插廣告
            news_time = datetime.strptime(dt, '%Y/%m/%d %H:%M')
        except:
            print('skip this item:', dt)
            print(link)
            continue

        dates.append(news_time.date())

        # item id
        t = datetime.strptime(dt, '%Y/%m/%d %H:%M')
        tstr = t.strftime("%Y%m%d")

        item_id.append(news_links[i] + "_" + tstr + "_" + str(serial_no))
        serial_no += 1

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
            #print(photo_link)
        except:
            photo_link = ''

        photo_links.append(photo_link)

        # There are some words we don't like to analyze, They should be removed.
        page = BeautifulSoup(requests.get( link ).text,'lxml')
        cont = page.find('div',{'class':"paragraph"}).text
        cont = re.sub('（中央社記者\w*）','', cont) # 去除不要的文字
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        cont = re.sub('（中央社\w*）','', cont) 
    
        contents.append(cont)
        time.sleep(2)  # 遵守爬蟲禮節，請小睡一下

# Save data

In [15]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [16]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,aipl_20210315_1,2021-03-15,政治,遭限制帳號部分功能 潘忠政：接受臉書機制,珍愛藻礁公投連署領銜人潘忠政遭臉書限制帳號部分功能，潘忠政今天表示，他非常支持同志的行動，相...,https://www.cna.com.tw/news/aipl/202103150283....,https://imgcdn.cna.com.tw/www/WebPhotos/400/20...
1,aipl_20210315_2,2021-03-15,政治,共軍2架運8擾台 空軍兵力應對,空軍今天發布共機動態，1架運8技偵機、1架運8反潛機下午侵犯台灣西南防空識別區（ADIZ ）...,https://www.cna.com.tw/news/aipl/202103150331....,


In [17]:
df.shape

(217, 7)

In [21]:
df.content[0]

'珍愛藻礁公投連署領銜人潘忠政遭臉書限制帳號部分功能，潘忠政今天表示，他非常支持同志的行動，相關文章其實已不存在，但無法阻止他人以此影射，他也接受臉書的相關機制。潘忠政下午出席立法院社會福利及衛生環境委員會，到桃園市觀音區考察觀塘工業區開發及第三天然氣接收站對大潭藻礁生態系統影響後接受記者聯訪表示，他發現這部分（疑似分享性別歧視言論）其實是維基百科上的內容，相關內容原貼已被刪除，這篇文章其實已不存在。潘忠政指出，有人影射他對同志不友善，其實他一直都非常支持同志的行動，也上街頭一起遊行，不斷告訴周圍的人這個新的價值觀，如果他人要以此影射，自己也沒辦法阻止。潘忠政說，他對於臉書（Facebook）認為這些文字不雅的相關機制也接受，他目前也不需要直播或做廣告，這件事情需要的話可以討論，不必造成社會尖銳的對立。潘忠政的臉書帳號成為已限制帳號，30天內不能直播、也不能發廣告。臉書今天回應，潘忠政因違反社群守則，已限制潘忠政帳號部分功能。'

In [18]:
df.to_csv("cna_news_200.csv", sep="|", index=False)